<a href="https://colab.research.google.com/github/nithinreganti/Neighnourhood-Ranking---Third-Estate-Analytics-Buffalo-NY---Capstone-Project-AWS-EC2-/blob/master/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
files = ['/content/drive/My Drive/Gentrification/Parking_Summonses .csv',
         '/content/drive/My Drive/Gentrification/Housing_Court_Cases.csv',
         '/content/drive/My Drive/Gentrification/Business_Licesnces_Updated_Description.xlsx',
         '/content/drive/My Drive/Gentrification/Permits.csv',
         '/content/drive/My Drive/Gentrification/311_Service_Requests.csv'
        ]

years = [2015, 2016, 2017, 2018]

In [0]:
def get_parking_df(year, parking):
    parking = parking[['VIOLATION DESCRIPTION', 'CENSUS TRACT', 'SUMMONS DATE']]
    parking = parking[parking['CENSUS TRACT'] != 'UNKNOWN']
    parking['CENSUS TRACT'] = parking['CENSUS TRACT'].astype(float)
    parking['SUMMONS DATE'] = pd.to_datetime(parking['SUMMONS DATE'])
    parking = parking[parking['SUMMONS DATE'].dt.year == year]
    parking = parking[['VIOLATION DESCRIPTION', 'CENSUS TRACT']]
    parking = pd.get_dummies(parking)
    parking.dropna(inplace=True)
    print('PARKING ' +str(year)+ ' CT: ', parking['CENSUS TRACT'].nunique())
    print('PARKING ' +str(year)+ ' Missing Values: ', parking.isnull().sum().sum())
    return parking

def get_court_df(year, court):
    court = court[['CENSUS TRACT', 'RESOLUTION', 'LASTACTION']]
    court = court[court['CENSUS TRACT'] != 'UNKNOWN']
    court['CENSUS TRACT'] = court['CENSUS TRACT'].astype(float)
    court['LASTACTION'] = court['LASTACTION'].apply(lambda x: x.split(' A')[0])
    court['LASTACTION'] = pd.to_datetime(court['LASTACTION'])
    court = court[court['LASTACTION'].dt.year == year]
    court = court[['CENSUS TRACT', 'RESOLUTION']]
    court = pd.get_dummies(court)
    court.dropna(inplace=True)
    print('COURT ' +str(year)+ ' CT: ', court['CENSUS TRACT'].nunique())
    print('COURT ' +str(year)+ ' Missing Values: ', court.isnull().sum().sum())
    return court
    
def get_license_df(year, license):
    license = license[['CENSUS TRACT', 'DESCRIPT', 'STATUSDTTM']]
    license = license[license['CENSUS TRACT'] != 'UNKNOWN']
    license['CENSUS TRACT'] = license['CENSUS TRACT'].astype(float)
    license['STATUSDTTM'] = pd.to_datetime(license['STATUSDTTM'])
    license = license[license['STATUSDTTM'].dt.year == year]
    license = license[['CENSUS TRACT', 'DESCRIPT']]
    license = pd.get_dummies(license)
    license.dropna(inplace=True)
    print('LICENSE ' +str(year)+ ' CT: ', license['CENSUS TRACT'].nunique())
    print('LICENSE ' +str(year)+ ' Missing Values: ', license.isnull().sum().sum())
    return license

def get_permit_df(year, permit):
    permit = permit[['PERMIT TYPE', 'CENSUS TRACT', 'ISSUED']]
    permit = permit[permit['CENSUS TRACT'] != 'UNKNOWN']
    permit['ISSUED'] = pd.to_datetime(permit['ISSUED'])
    permit['CENSUS TRACT'] = permit['CENSUS TRACT'].astype(float)
    permit = permit[permit['ISSUED'].dt.year == year]
    permit = permit[['PERMIT TYPE', 'CENSUS TRACT']]
    permit = pd.get_dummies(permit)
    permit.dropna(inplace=True)
    permit = permit.sample(n=1000, random_state=14)
    print('PERMIT ' +str(year)+ ' CT: ', permit['CENSUS TRACT'].nunique())
    print('PERMIT ' +str(year)+ ' Missing Values: ', permit.isnull().sum().sum())
    return permit

def get_service_df(year, service):
    service = service[['TYPE', 'CENSUS TRACT', 'CLOSED DATE']]
    service = service[service['CENSUS TRACT'] != 'UNKNOWN']
    service['CENSUS TRACT'] = service['CENSUS TRACT'].astype(float)
    service['CLOSED DATE'] = pd.to_datetime(service['CLOSED DATE'])
    service = service[service['CLOSED DATE'].dt.year == year]
    service = service[['TYPE', 'CENSUS TRACT']]
    service = pd.get_dummies(service)
    service.dropna(inplace=True)
    service = service.sample(n=1000, random_state=27)
    print('SERVICE ' +str(year)+ ' CT: ', service['CENSUS TRACT'].nunique())
    print('SERVICE ' +str(year)+ ' Missing Values: ', service.isnull().sum().sum())
    return service
    
def merge_df(year, files):
    parking = pd.read_csv(files[0])
    print('Processing parking data for ' +str(year)+ '....')
    parking_df = get_parking_df(year, parking)
    del parking
    
    court = pd.read_csv(files[1])
    print('Processing court data for ' +str(year)+ '....')
    court_df = get_court_df(year, court)
    del court
    
    license = pd.read_excel(files[2])
    print('Processing license data for ' +str(year)+ '....')  
    license_df = get_license_df(year, license)
    del license
    
    permit = pd.read_csv(files[3])
    print('Processing permit data for ' +str(year)+ '....')
    permit_df = get_permit_df(year, permit)
    del permit
    
    service = pd.read_csv(files[4])
    print('Processing service data for ' +str(year)+ '....')
    service_df = get_service_df(year, service)
    del service
    
    df = pd.merge(parking_df, court_df, 'left', 'CENSUS TRACT')
    df = pd.merge(df, license_df, 'left', 'CENSUS TRACT')
    df = df.sample(n=100000)
    df = pd.merge(df, service_df, 'left', 'CENSUS TRACT')
    df = df.sample(n=100000)
    df = pd.merge(df, permit_df, 'left', 'CENSUS TRACT')
    
    print('Merged 5 datasets.')
    print('Shape: ', df.shape)
    print('Total CTs: ', df['CENSUS TRACT'].nunique())
    return df

def build_model(df):
    X = df.drop(columns = ['Rank', 'CENSUS TRACT'])
    y = df[['Rank', 'CENSUS TRACT']]
    del df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    rank_df = y_test
    y_train = y_train['Rank']
    y_test = y_test['Rank']
    
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    
    predictions = ridge.predict(X_test)
    print('MAE: ', mean_absolute_error(y_test, predictions))
    
    cdf = pd.DataFrame({'Features': X_train.columns,
                        'Coefficients': ridge.coef_
                       }).sort_values(by='Coefficients', ascending=False)
    
    rank_df['PredictedRank'] = predictions
    
    return cdf, rank_df

In [21]:
year = 2018
df = merge_df(year, files)
rank = pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice_2018.csv')
rank = rank[rank['CENSUS TRACT'] != 'UNKNOWN']
rank['CENSUS TRACT'] = rank['CENSUS TRACT'].astype(float)
df = pd.merge(df, rank, 'left', 'CENSUS TRACT') 
df.fillna(value=0, inplace=True)
df.drop(columns=['Price/sqft'], inplace=True)
print('After applying rank to merged dataset.')
print('Shape: ', df.shape)
print('Total CTs: ', df['CENSUS TRACT'].nunique())
print('Total Missing Values: ', df.isnull().sum().sum())
cdf_2018, rank_2018 = build_model(df)
#         cdf_2018.to_csv('F:/Capstone/Results/cdf2018.csv', index=False)
#         rank_2018.to_csv('F:/Capstone/Results/rank2018.csv', index=False)
print('\n<------------------------------------------------->\n')
del df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing parking data for 2018....
PARKING 2018 CT:  79
PARKING 2018 Missing Values:  0
Processing court data for 2018....


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


COURT 2018 CT:  72
COURT 2018 Missing Values:  0
Processing license data for 2018....
LICENSE 2018 CT:  62
LICENSE 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing permit data for 2018....
PERMIT 2018 CT:  78
PERMIT 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,13,15,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing service data for 2018....
SERVICE 2018 CT:  79
SERVICE 2018 Missing Values:  0
Merged 5 datasets.
Shape:  (1978085, 238)
Total CTs:  77
After applying rank to merged dataset.
Shape:  (1978085, 239)
Total CTs:  77
Total Missing Values:  0
MAE:  8.999789089874136

<------------------------------------------------->



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
year == 2017
df = merge_df(year, files)
rank = pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice_2017.csv')
rank = rank[rank['CENSUS TRACT'] != 'UNKNOWN']
rank['CENSUS TRACT'] = rank['CENSUS TRACT'].astype(float)
df = pd.merge(df, rank, 'left', 'CENSUS TRACT') 
df.fillna(value=0, inplace=True)
df.drop(columns=['Price/sqft'], inplace=True)
print('After applying rank to merged dataset.')
print('Shape: ', df.shape)
print('Total CTs: ', df['CENSUS TRACT'].nunique())
print('Total Missing Values: ', df.isnull().sum().sum())
cdf_2017, rank_2017 = build_model(df)
#         cdf_2017.to_csv('F:/Capstone/Results/cdf2017.csv', index=False)
#         rank_2017.to_csv('F:/Capstone/Results/rank2017.csv', index=False)
print('\n<------------------------------------------------->\n')
del df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing parking data for 2018....
PARKING 2018 CT:  79
PARKING 2018 Missing Values:  0
Processing court data for 2018....


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


COURT 2018 CT:  72
COURT 2018 Missing Values:  0
Processing license data for 2018....
LICENSE 2018 CT:  62
LICENSE 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing permit data for 2018....
PERMIT 2018 CT:  78
PERMIT 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,13,15,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing service data for 2018....
SERVICE 2018 CT:  79
SERVICE 2018 Missing Values:  0
Merged 5 datasets.
Shape:  (1973981, 238)
Total CTs:  77
After applying rank to merged dataset.
Shape:  (1973981, 239)
Total CTs:  77
Total Missing Values:  0
MAE:  8.587859685495332

<------------------------------------------------->



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
year == 2016
df = merge_df(year, files)
rank = pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice_2016.csv')
rank = rank[rank['CENSUS TRACT'] != 'UNKNOWN']
rank['CENSUS TRACT'] = rank['CENSUS TRACT'].astype(float)
df = pd.merge(df, rank, 'left', 'CENSUS TRACT') 
df.fillna(value=0, inplace=True)
df.drop(columns=['Price/sqft'], inplace=True)
print('After applying rank to merged dataset.')
print('Shape: ', df.shape)
print('Total CTs: ', df['CENSUS TRACT'].nunique())
print('Total Missing Values: ', df.isnull().sum().sum())
cdf_2016, rank_2016 = build_model(df)
#         cdf_2016.to_csv('F:/Capstone/Results/cdf2016.csv', index=False)
#         rank_2016.to_csv('F:/Capstone/Results/rank2016.csv', index=False)
print('\n<------------------------------------------------->\n')
del df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing parking data for 2018....
PARKING 2018 CT:  79
PARKING 2018 Missing Values:  0
Processing court data for 2018....


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


COURT 2018 CT:  72
COURT 2018 Missing Values:  0
Processing license data for 2018....
LICENSE 2018 CT:  62
LICENSE 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing permit data for 2018....
PERMIT 2018 CT:  78
PERMIT 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,13,15,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing service data for 2018....
SERVICE 2018 CT:  79
SERVICE 2018 Missing Values:  0
Merged 5 datasets.
Shape:  (1979072, 238)
Total CTs:  77
After applying rank to merged dataset.
Shape:  (1979072, 239)
Total CTs:  77
Total Missing Values:  0
MAE:  8.608251723340253

<------------------------------------------------->



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
year == 2015
df = merge_df(year, files)
rank = pd.read_csv('/content/drive/My Drive/Gentrification/census_rank_Assessmentprice_2015.csv')
rank = rank[rank['CENSUS TRACT'] != 'UNKNOWN']
rank['CENSUS TRACT'] = rank['CENSUS TRACT'].astype(float)
df = pd.merge(df, rank, 'left', 'CENSUS TRACT') 
df.fillna(value=0, inplace=True)
df.drop(columns=['Price/sqft'], inplace=True)
print('After applying rank to merged dataset.')
print('Shape: ', df.shape)
print('Total CTs: ', df['CENSUS TRACT'].nunique())
print('Total Missing Values: ', df.isnull().sum().sum())
cdf_2015, rank_2015 = build_model(df)
#         cdf_2015.to_csv('F:/Capstone/Results/cdf2015.csv', index=False)
#         rank_2015.to_csv('F:/Capstone/Results/rank2015.csv', index=False)
print('\n<------------------------------------------------->\n')
del df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing parking data for 2018....
PARKING 2018 CT:  79
PARKING 2018 Missing Values:  0
Processing court data for 2018....


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


COURT 2018 CT:  72
COURT 2018 Missing Values:  0
Processing license data for 2018....
LICENSE 2018 CT:  62
LICENSE 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing permit data for 2018....
PERMIT 2018 CT:  78
PERMIT 2018 Missing Values:  0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,13,15,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processing service data for 2018....
SERVICE 2018 CT:  79
SERVICE 2018 Missing Values:  0
Merged 5 datasets.
Shape:  (1977971, 238)
Total CTs:  77
After applying rank to merged dataset.
Shape:  (1977971, 239)
Total CTs:  77
Total Missing Values:  0
MAE:  9.211493549467946

<------------------------------------------------->



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
 cdf_2018.head()

,Features,Coefficients
44,DESCRIPT_Lodging House,37.586352
161,TYPE_Recycling Tote Abandon Pickup (Req_Serv),33.944710
135,TYPE_PVB Single Meter (Req_Serv),33.189995
57,DESCRIPT_Wholesale Junk,33.081078
175,TYPE_Signal Other Issue (Req_Serv),32.966060


In [26]:
 cdf_2018.tail()

,Features,Coefficients
232,PERMIT TYPE_SWIM POOL,-17.165109
85,TYPE_Damaged Street Light Pole (Req_Serv),-20.296996
81,TYPE_Curb - Metal Protruding (Req_Serv),-22.517122
109,TYPE_Graffiti Private Property (Req_Serv),-24.544237
61,TYPE_Assessment Issue (Req_Serv),-31.590799


In [27]:
 cdf_2018[cdf_2018['Features'].str.contains('DESCRIPT')].head()

,Features,Coefficients
44,DESCRIPT_Lodging House,37.586352
57,DESCRIPT_Wholesale Junk,33.081078
18,VIOLATION DESCRIPTION_NO VIOLATION,30.694348
54,DESCRIPT_Stationary Peddler,25.529779
55,DESCRIPT_Tire Handler,16.608212


In [28]:
 cdf_2018[cdf_2018['Features'].str.contains('DESCRIPT')].tail()

,Features,Coefficients
48,DESCRIPT_Restaurant,-11.262424
43,DESCRIPT_Live Music / No Dancing,-11.732416
41,DESCRIPT_Bowling Alley,-12.974906
47,DESCRIPT_Parking Lot,-15.542035
53,DESCRIPT_Sidewalk Cafe,-16.847349


In [29]:
 rank_2018.sample(20)

,Rank,CENSUS TRACT,PredictedRank
303077,1,165.00,1.178758
1501865,1,165.00,2.005209
1228033,18,49.00,27.556526
247824,1,165.00,3.503998
1001521,61,31.00,41.456869
890371,18,49.00,22.912417
388020,15,52.01,24.914056
1076583,7,68.00,20.212401
1151925,1,165.00,8.586965
1731867,37,47.00,37.995014
